In [1]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')

nan_percentage = df.isnull().mean()

# 3. Izberi samo tiste stolpce, ki imajo MANJ kot 50% NaN vrednosti
# (Prag lahko poljubno spremeniš, npr. 0.3 za še strožje čiščenje)
relevant_columns = nan_percentage[nan_percentage < 0.5].index.tolist()

print(f"Obdržali bomo {len(relevant_columns)} stolpcev od začetnih {len(df.columns)}.")
print("Obdržani stolpci:", relevant_columns)

Obdržali bomo 47 stolpcev od začetnih 197.
Obdržani stolpci: ['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'datacenter', 'cluster_id', 'vault_id', 'pod_id', 'pod_slot_num', 'is_legacy_format', 'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized', 'smart_2_raw', 'smart_3_normalized', 'smart_3_raw', 'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw', 'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_10_normalized', 'smart_10_raw', 'smart_12_normalized', 'smart_12_raw', 'smart_192_normalized', 'smart_192_raw', 'smart_193_normalized', 'smart_193_raw', 'smart_194_normalized', 'smart_194_raw', 'smart_196_normalized', 'smart_196_raw', 'smart_197_normalized', 'smart_197_raw', 'smart_198_normalized', 'smart_198_raw', 'smart_199_normalized', 'smart_199_raw', 'smart_240_normalized', 'smart_240_raw']
